In [1]:
# -*- coding: utf-8 -*-
# 파일명: train_models_v2.py (새로운 데이터셋으로 모델 재학습)
# 실행 환경: 64비트 Python (myenv)
# 필요 라이브러리: tensorflow, pandas, numpy, scikit-learn

import pandas as pd
import numpy as np
import os
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam # Adam 임포트 명시
import warnings
import gc
import traceback
warnings.filterwarnings('ignore')

# --- 1. 설정값 정의 ---
# ★★★ 경로 수정: 새로 가공된 데이터 및 새 모델 저장 경로 ★★★
PROCESSED_DATA_PATH = '/content/drive/MyDrive/processed_stock_data_full_v2/processed_parquet'
NEW_MODEL_SAVE_PATH = '/content/drive/MyDrive/lstm_models_per_stock_v2'

# 모델 하이퍼파라미터 (이전과 동일하게 유지하거나 필요시 조정)
TIME_STEPS = 20
TRAIN_RATIO = 0.7
VALID_RATIO = 0.15
# TEST_RATIO = 1 - TRAIN_RATIO - VALID_RATIO
EPOCHS = 50 # 최대 에포크 (EarlyStopping 사용)
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# ★★★ 사용할 피처 컬럼 목록 (데이터 처리 스크립트와 동일하게) ★★★
# process_and_save_data_with_scalers.py 에서 최종 Parquet에 저장된 컬럼 중 모델 입력으로 사용할 것들
# _norm 접미사가 붙은 정규화된 피처와, 정규화 안 된 피처(is_month_end) 등
FEATURE_COLUMNS_FOR_MODEL = [
    'open_norm', 'high_norm', 'low_norm', 'close_norm', 'volume_norm', 'amount_norm',
    'SMA_5_norm', 'SMA_20_norm', 'SMA_60_norm', 'SMA_120_norm', 'RSI_14_norm',
    'MACD_12_26_9_norm', 'MACDs_12_26_9_norm', 'MACDh_12_26_9_norm',
    'BBL_20_2.0_norm', 'BBM_20_2.0_norm', 'BBU_20_2.0_norm', 'BBB_20_2.0_norm', 'BBP_20_2.0_norm',
    'ATRr_14_norm', 'OBV_norm', 'STOCHk_14_3_3_norm', 'STOCHd_14_3_3_norm',
    'PBR_norm', 'PER_norm', 'USD_KRW_norm', # 이 값들은 Parquet 파일에 _norm으로 저장됨
    'is_month_end' # 이 값은 원본 그대로 Parquet 파일에 저장됨
]
TARGET_COLUMN_FOR_MODEL = 'close_norm' # 예측 대상 컬럼 (정규화된 종가)

NUM_FEATURES_FOR_MODEL = len(FEATURE_COLUMNS_FOR_MODEL)

LIMIT_STOCKS_TO_TRAIN = None # ★★★ None이면 전체 학습, 테스트 시 5 등으로 설정 ★★★
# ---------------------------------------------------------

# --- 2. 모델 저장 경로 생성 ---
os.makedirs(NEW_MODEL_SAVE_PATH, exist_ok=True)
print(f"가공된 데이터 경로: {PROCESSED_DATA_PATH}")
print(f"새 모델 저장 경로: {NEW_MODEL_SAVE_PATH}")

# --- 3. 학습할 데이터 파일 목록 가져오기 ---
try:
    parquet_files = [f for f in os.listdir(PROCESSED_DATA_PATH) if f.endswith('.parquet')]
    if not parquet_files:
        raise FileNotFoundError(f"Parquet 파일이 '{PROCESSED_DATA_PATH}' 경로에 없습니다.")
    parquet_files.sort() # 일관된 순서로 처리
    print(f"총 {len(parquet_files)}개의 Parquet 파일 확인.")
    if LIMIT_STOCKS_TO_TRAIN is not None and LIMIT_STOCKS_TO_TRAIN > 0:
        parquet_files = parquet_files[:LIMIT_STOCKS_TO_TRAIN]
        print(f"★★ 테스트를 위해 {LIMIT_STOCKS_TO_TRAIN}개 종목만 학습합니다. ★★")
except FileNotFoundError as e:
    print(f"오류: {e}. 데이터 경로를 확인하세요.")
    exit()
except Exception as e:
    print(f"파일 목록 로딩 중 오류 발생: {e}")
    exit()

# --- 4. 시퀀스 생성 함수 ---
def create_sequences(data, target, time_steps=TIME_STEPS): # time_steps 전역변수 사용
    Xs, ys = [], []
    # feature_data는 이미 선택된 컬럼들로 구성된 NumPy 배열이라고 가정
    for i in range(len(data) - time_steps):
        Xs.append(data[i:(i + time_steps)])
        ys.append(target[i + time_steps]) # time_steps 이후의 target 값
    return np.array(Xs), np.array(ys)

# --- 5. LSTM 모델 정의 함수 ---
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=1)) # 출력층
    return model

# --- 6. 전체 종목 모델 학습 루프 ---
print(f"\n--- {len(parquet_files)}개 종목 모델 재학습 시작 ---")
total_train_start_time = time.time()
models_trained_count = 0
models_failed_count = 0

for i, file_name in enumerate(parquet_files):
    ticker = file_name.replace('.parquet', '')
    data_file_path = os.path.join(PROCESSED_DATA_PATH, file_name)
    model_save_file_path = os.path.join(NEW_MODEL_SAVE_PATH, f"{ticker}.keras")

    print(f"\n[{i+1}/{len(parquet_files)}] Ticker: {ticker} 학습 시작...")
    ticker_train_start_time = time.time()

    try:
        # 6.1 데이터 로드
        df_loaded = pd.read_parquet(data_file_path)

        # 사용할 피처 및 타겟 컬럼 존재 확인
        missing_cols = [col for col in FEATURE_COLUMNS_FOR_MODEL if col not in df_loaded.columns]
        if missing_cols:
            print(f"  >> 오류: 필요한 피처 컬럼 누락: {missing_cols}. 건너<0xEB><0x8A>니다.")
            models_failed_count += 1
            continue
        if TARGET_COLUMN_FOR_MODEL not in df_loaded.columns:
            print(f"  >> 오류: 타겟 컬럼 '{TARGET_COLUMN_FOR_MODEL}' 없음. 건너<0xEB><0x8A>니다.")
            models_failed_count += 1
            continue

        # 피처 및 타겟 데이터 추출
        feature_data = df_loaded[FEATURE_COLUMNS_FOR_MODEL].values.astype(np.float32)
        target_data = df_loaded[TARGET_COLUMN_FOR_MODEL].values.astype(np.float32)

        # NaN 값 확인 (이론적으로는 이전 단계에서 dropna 했으므로 없어야 함)
        if np.isnan(feature_data).any() or np.isnan(target_data).any():
             print(f"  >> 경고: 로드된 데이터에 NaN 존재. nan_to_num 처리 시도.")
             feature_data = np.nan_to_num(feature_data, nan=0.0) # 임시로 0으로 대체
             target_data = np.nan_to_num(target_data, nan=0.0)

        # 6.2 시퀀스 생성
        X, y = create_sequences(feature_data, target_data) # TIME_STEPS는 전역변수 사용

        if len(X) < (TIME_STEPS * 2): # 분할에 충분한 시퀀스 길이 확인 (최소 Train/Valid)
             print(f"  >> 경고: 시퀀스 생성 후 데이터 길이 부족 ({len(X)}). 건너<0xEB><0x8A>니다.")
             models_failed_count += 1
             continue

        # 6.3 데이터 분할 (Train/Validation/Test) - 시계열 특성 유지
        n_total = len(X)
        n_train = int(n_total * TRAIN_RATIO)
        n_valid = int(n_total * VALID_RATIO)
        # n_test = n_total - n_train - n_valid # 테스트는 학습 후 별도 평가

        X_train, y_train = X[:n_train], y[:n_train]
        X_valid, y_valid = X[n_train : n_train + n_valid], y[n_train : n_train + n_valid]
        # X_test, y_test = X[n_train + n_valid:], y[n_train + n_valid:] # 테스트셋은 저장된 모델로 나중에 평가

        if len(X_train) == 0 or len(X_valid) == 0 or X_train.shape[1] != TIME_STEPS or X_train.shape[2] != NUM_FEATURES_FOR_MODEL:
             print(f"  >> 경고: 학습 또는 검증 데이터 형태 오류 또는 부족. 건너<0xEB><0x8A>니다.")
             print(f"     Train shape: {X_train.shape if len(X_train)>0 else 'Empty'}, Valid shape: {X_valid.shape if len(X_valid)>0 else 'Empty'}")
             print(f"     Expected TIME_STEPS: {TIME_STEPS}, Expected NUM_FEATURES: {NUM_FEATURES_FOR_MODEL}")
             models_failed_count += 1
             continue

        print(f"  >> 데이터 분할: Train={len(X_train)}, Validation={len(X_valid)}")
        print(f"  >> 입력 데이터 형태 (첫 샘플): {X_train[0].shape}") # (TIME_STEPS, NUM_FEATURES_FOR_MODEL)

        # 6.4 모델 구축 및 컴파일
        model = build_lstm_model(input_shape=(TIME_STEPS, NUM_FEATURES_FOR_MODEL))
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')

        # 6.5 모델 학습
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint(model_save_file_path, monitor='val_loss', verbose=0, save_best_only=True)

        print(f"  >> 모델 학습 시작 (최대 {EPOCHS} 에포크)...")
        history = model.fit(
            X_train, y_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_data=(X_valid, y_valid),
            callbacks=[early_stopping, model_checkpoint],
            verbose=0 # 학습 로그 최소화
        )

        # 6.6 학습 완료 및 결과 요약
        if 'val_loss' in history.history and history.history['val_loss']:
             best_epoch = np.argmin(history.history['val_loss']) + 1
             min_val_loss = np.min(history.history['val_loss'])
             print(f"  >> 모델 학습 완료. 최적 에포크: {best_epoch}, 최소 검증 손실(MSE): {min_val_loss:.6f}")
        else:
             print(f"  >> 모델 학습 완료 (기록 부족 또는 없음).") # EarlyStopping으로 1 epoch 전에 멈춘 경우 등
             min_val_loss = float('inf')


        if os.path.exists(model_save_file_path) and min_val_loss != float('inf'): # 저장되었고 유효한 학습이 이루어졌다면
             print(f"  >> 최적 모델 저장됨: {model_save_file_path}")
             models_trained_count += 1
        else:
             print(f"  >> 정보: 최적 모델 파일이 저장되지 않았거나 학습이 유효하지 않을 수 있습니다 (경로: {model_save_file_path}, 최소 val_loss: {min_val_loss}).")
             # 실패 카운트보다는, 학습은 시도했으므로 별도 처리 가능

        # 메모리 관리
        del df_loaded, feature_data, target_data, X, y, X_train, y_train, X_valid, y_valid, model, history
        tf.keras.backend.clear_session()
        gc.collect()

    except Exception as e:
        print(f"  >> 오류: 티커 {ticker} 처리 중 예외 발생: {e}")
        traceback.print_exc()
        models_failed_count += 1
        continue
    finally:
        ticker_train_end_time = time.time()
        print(f"  Ticker: {ticker} 학습 시간: {ticker_train_end_time - ticker_train_start_time:.2f} 초")

# --- 최종 결과 출력 ---
total_train_end_time = time.time()
print("\n--- 전체 종목 모델 재학습 완료 ---")
print(f"총 소요 시간: {(total_train_end_time - total_train_start_time) / 60:.2f} 분")
print(f"성공적으로 학습/저장된 모델 수: {models_trained_count}")
print(f"실패 또는 건너뛴 종목 수: {models_failed_count}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  >> 모델 학습 완료. 최적 에포크: 9, 최소 검증 손실(MSE): 0.002427
  >> 최적 모델 저장됨: /content/drive/MyDrive/lstm_models_per_stock_v2/000640.keras
  Ticker: 000640 학습 시간: 12.07 초

[40/663] Ticker: 000650 학습 시작...
  >> 데이터 분할: Train=821, Validation=175
  >> 입력 데이터 형태 (첫 샘플): (20, 27)
  >> 모델 학습 시작 (최대 50 에포크)...
  >> 모델 학습 완료. 최적 에포크: 2, 최소 검증 손실(MSE): 0.000926
  >> 최적 모델 저장됨: /content/drive/MyDrive/lstm_models_per_stock_v2/000650.keras
  Ticker: 000650 학습 시간: 8.77 초

[41/663] Ticker: 000660 학습 시작...
  >> 데이터 분할: Train=821, Validation=175
  >> 입력 데이터 형태 (첫 샘플): (20, 27)
  >> 모델 학습 시작 (최대 50 에포크)...
  >> 모델 학습 완료. 최적 에포크: 2, 최소 검증 손실(MSE): 0.026661
  >> 최적 모델 저장됨: /content/drive/MyDrive/lstm_models_per_stock_v2/000660.keras
  Ticker: 000660 학습 시간: 9.19 초

[42/663] Ticker: 000670 학습 시작...
  >> 데이터 분할: Train=814, Validation=174
  >> 입력 데이터 형태 (첫 샘플): (20, 27)
  >> 모델 학습 시작 (최대 50 에포크)...
  >> 모델 학습 완료. 최적 에포크: 21, 최소 검증 손실(MSE): 0.000426
  >> 최적 모델 저장됨: /content/drive/MyDri